## Chapter 2-5, 2강 TTS(텍스트→음성) 기초와 보이스 클로닝 — Coqui TTS

- 목표: TTS 기본 파이프라인 이해, 멀티스피커/멀티언어 모델 사용, 간단 보이스 클로닝 실습
- 데이터: 짧은 텍스트 문장과 참조 화자 오디오(30~60초)
- 규칙(강의용): 시각화는 `matplotlib`만 사용, 불필요한 외부 의존 최소화


### 구성 (Overview)
1) TTS 개요 & 파이프라인(보이스 클로닝 포함)
2) 환경 준비(라이브러리/폰트/경고) 및 `DEVICE` 출력
3) 데이터 준비: 참조 화자 오디오 전처리(16kHz/모노)
4) Google Cloud TTS — 설치·샘플
5) 텍스트 전처리: 정규화/문장 분할의 영향 실험
6) 합성 속도/길이 벤치마크(간단)
7) 시각화: 파형/멜스펙 + 청취 포인트
8) 마무리 및 과제(응용/평가)

### 1. TTS 개요 — 파이프라인과 보이스 클로닝

- 텍스트→음성(TTS) 파이프라인은 일반적으로 다음 단계를 거칩니다.
  1) 텍스트 전처리(Text Normalization, 문장 분할)
  2) 발음/음운 변환(언어별 G2P/발음사전 또는 End-to-End)
  3) 어쿠스틱 모델(스펙트로그램/음성 특성 예측)
  4) 보코더(Vocoder, 예: HiFi-GAN/BigVGAN 계열)로 파형 생성
- 최근 모델은 2–4 단계를 End-to-End로 통합하거나 토큰/디퓨전 기반을 사용하며, 다국어·다화자·제로샷 보이스 클로닝을 지원하는 모델들이 존재합니다.

#### 보이스 클로닝(Zero-shot Speaker Adaptation)
- 짧은 참조 음성(30–60초)으로 화자 임베딩을 추출해, 새로운 텍스트를 해당 화자 톤으로 합성
- 참고: 참조 음성의 언어/발음, 녹음 품질, 배경잡음이 클로닝 품질에 큰 영향

#### 품질에 영향을 주는 요인
- 텍스트: 숫자·기호·영문 혼용 → 정규화 필요, 문장부호로 억양 유도
- 오디오: 참조 음성의 SNR, 마이크 품질, 리버브/노이즈
- 모델: 언어 지원 범위, 학습 데이터 커버리지, 보코더 품질



### 0. 환경 준비 및 라이브러리 임포트


In [1]:
import os, warnings
import numpy as np
import matplotlib.pyplot as plt
import torch, torchaudio

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=r"Glyph.*missing from font.*", category=UserWarning)

warnings.filterwarnings("ignore")
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', DEVICE)



Device: cpu


### 1. 데이터 준비 — 오디오 16k/모노
- 녹음/샘플 WAV를 16kHz/모노로 통일해 TTS 입출력 품질 비교와 재현성을 확보합니다.



In [2]:
def ensure_mono_16k(path: str, out_path: str = 'tmp_16k_mono.wav') -> str:
    wav, sr = torchaudio.load(path)
    if wav.size(0) > 1:
        wav = wav.mean(dim=0, keepdim=True)
    if sr != 16000:
        wav = torchaudio.transforms.Resample(sr, 16000)(wav)
    torchaudio.save(out_path, wav, 16000)
    return out_path

# 예시
# ensure_mono_16k('input.wav', 'tmp_16k_mono.wav')



In [20]:
ensure_mono_16k('./data/speech/1_0000.wav', 'tmp_16k_mono.wav')

'tmp_16k_mono.wav'

### 2-1. 텍스트 전처리와 문장 분할
- **텍스트 정규화(Text Normalization)**: 숫자, 기호, 단위, 영어 혼용을 **읽기 쉬운 형태**로 바꾸어 발음을 안정화합니다.
  - 예) "1kg" → "1 킬로그램", "A/S" → "에이에스"
- **문장 분할(Sentence Splitting)**: 문장부호(`.?!`, 쉼표)로 문장을 짧게 쪼개면 억양이 안정적이고 지연이 줄어듭니다.
- 실습: 동일 문장을 분할 유무/부호 유무에 따라 합성 품질(억양/호흡)을 비교합니다.



In [4]:
# 텍스트 정규화 & 문장 분할 유틸
import re
from typing import List

def normalize_korean_text(text: str) -> str:
    """
    간단한 한글 중심 텍스트 정규화 예시.
    - 숫자·단위/기호 일부를 읽기 쉬운 형태로 치환
    - 과도한 공백 축소
    참고: 실제 서비스는 언어별 규칙과 도메인 사전을 체계화해야 함
    """
    rules = [
        (r"\bA/S\b", "에이에스"),
        (r"\bAS\b", "에이에스"),
        (r"\bkg\b", "킬로그램"),
        (r"\bkm\b", "킬로미터"),
    ]
    out = text
    for pat, rep in rules:
        out = re.sub(pat, rep, out, flags=re.IGNORECASE)
    out = re.sub(r"\s+", " ", out).strip()
    return out

def split_sentences(text: str) -> List[str]:
    """
    문장부호(.?!;) 기준 간단 분할. 쉼표는 선택적으로 유지.
    """
    parts = re.split(r"([.!?;])", text)
    sents = []
    buf = ""
    for p in parts:
        if re.match(r"[.!?;]", p):
            buf += p
            sents.append(buf.strip())
            buf = ""
        else:
            buf += p
    if buf.strip():
        sents.append(buf.strip())
    return [s for s in sents if s]

# 데모용 문장
RAW_TEXT = "오늘 기온은 23.5도, 바람이 강합니다! 1kg짜리 샘플과 A/S 절차를 안내합니다."
print('정규화 전:', RAW_TEXT)
print('정규화 후:', normalize_korean_text(RAW_TEXT))
print('문장 분할:', split_sentences(normalize_korean_text(RAW_TEXT)))



정규화 전: 오늘 기온은 23.5도, 바람이 강합니다! 1kg짜리 샘플과 A/S 절차를 안내합니다.
정규화 후: 오늘 기온은 23.5도, 바람이 강합니다! 1kg짜리 샘플과 에이에스 절차를 안내합니다.
문장 분할: ['오늘 기온은 23.', '5도, 바람이 강합니다!', '1kg짜리 샘플과 에이에스 절차를 안내합니다.']


### 2-2. Google Cloud TTS — 개요
- 클라우드 API 기반 TTS로, 고품질/다양한 보이스를 제공하며 SDK로 쉽게 사용 가능합니다.
- 아래에 설치/설정 및 최소 동작 샘플 코드를 제공합니다.



In [5]:
# GCP 의존성 설치 보조 (필요 시 실행)
# - GCP TTS: google-cloud-texttospeech
try:
    ensure
except NameError:
    import importlib, sys, subprocess
    def ensure(pkg: str, pip_name: str | None = None):
        name = pip_name or pkg
        try:
            importlib.import_module(pkg)
        except Exception:
            print(f'Installing {name} ...')
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', name, '--quiet'])

# macOS에서 libsndfile이 필요할 수 있습니다: Homebrew → `brew install libsndfile`
ensure('google.cloud.texttospeech', 'google-cloud-texttospeech')
ensure('onnxruntime')
ensure('soundfile')



#### 2-2-2) Google Cloud TTS — 설치 및 설정
- 설치: `pip install google-cloud-texttospeech`
- GCP 프로젝트/결제 설정 → API 활성화 → 서비스 계정 키(JSON) 발급
- 인증: 환경 변수 설정 또는 코드 내에서 자격증명 로딩
  - macOS/zsh 예: `export GOOGLE_APPLICATION_CREDENTIALS="/absolute/path/key.json"`



In [14]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cp-100-187113-690d09617cf5.json"


In [16]:
# Google Cloud TTS 최소 동작 예제 (텍스트 → WAV)
# 요구: pip install google-cloud-texttospeech
from google.cloud import texttospeech
import base64
import wave


def gcp_tts_synthesize(text: str, out_wav: str = 'gcp_out.wav', language_code: str = 'ko-KR', voice_name: str | None = None, speaking_rate: float = 1.0):
    client = texttospeech.TextToSpeechClient()

    voice_params = dict(language_code=language_code)
    if voice_name:
        voice_params['name'] = voice_name
    voice = texttospeech.VoiceSelectionParams(**voice_params)

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16,
        speaking_rate=speaking_rate,
        sample_rate_hertz=24000
    )

    synthesis_input = texttospeech.SynthesisInput(text=text)
    response = client.synthesize_speech(
        input=synthesis_input,
        voice=voice,
        audio_config=audio_config
    )

    # LINEAR16 PCM → WAV 래핑
    pcm = response.audio_content
    with wave.open(out_wav, 'wb') as f:
        f.setnchannels(1)
        f.setsampwidth(2)  # 16-bit
        f.setframerate(24000)
        f.writeframes(pcm)
    return out_wav

# 실행 예시(주석 해제)
# gcp_tts_synthesize('안녕하세요. 구글 클라우드 TTS 데모입니다.', 'gcp_out.wav', language_code='ko-KR')



In [17]:
gcp_tts_synthesize('안녕하세요. 구글 클라우드 TTS 데모입니다.', 'gcp_out.wav', language_code='ko-KR')

E0000 00:00:1758549640.775566  201140 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


'gcp_out.wav'

#### 2-2-3) 결론/추천
- 고품질·다양한 보이스·운영 단순: Google Cloud TTS 권장
- 비용/네트워크 의존 고려: 문자 과금, 키 관리 필요



### 3. 합성 속도/길이 벤치마크(간단)
- 동일 문장 세트로 Piper CLI/GCP TTS 합성 시간을 비교합니다.



In [8]:
import time

def bench_gcp(texts: list[str], language_code: str = 'ko-KR'):
    rows = []
    for t in texts:
        t0 = time.perf_counter()
        gcp_tts_synthesize(t, 'bench_gcp.wav', language_code=language_code)
        rows.append({'engine': 'gcp', 'len': len(t), 'sec': round(time.perf_counter()-t0, 2)})
    return rows

# 예시
# texts = ['안녕하세요.', '오늘은 TTS 벤치마크를 진행합니다.', '문장이 길어질수록 시간이 늘어납니다. 적절한 길이로 분할하세요.']
# print(bench_gcp(texts, 'ko-KR'))



### 4. 시각화 — 파형/멜스펙과 청취 포인트
- 합성 결과를 파형과 멜 스펙트로그램으로 확인합니다.
- 억양/호흡/잡음(메탈릭) 체크 포인트를 함께 제시합니다.



In [19]:
def plot_wave_and_melspec(wav_path: str):
    """
    WAV 파일을 로드하여 파형과 멜 스펙트로그램을 시각화합니다.
    """
    if not os.path.exists(wav_path):
        raise FileNotFoundError('합성 파일을 찾을 수 없습니다: ' + wav_path)
    wav, sr = torchaudio.load(wav_path)
    if wav.size(0) > 1:
        wav = wav.mean(dim=0, keepdim=True)
    # 파형
    t = np.arange(wav.size(1)) / sr
    plt.figure(figsize=(12, 3))
    plt.plot(t, wav.squeeze(0).numpy(), color='steelblue')
    plt.title('Waveform')
    plt.xlabel('Time (s)')
    plt.tight_layout()
    plt.show()
    # 멜 스펙트로그램
    mel = torchaudio.transforms.MelSpectrogram(
        sample_rate=sr, n_fft=1024, hop_length=256, n_mels=80
    )(wav)
    mel_db = torchaudio.transforms.AmplitudeToDB()(mel)
    plt.figure(figsize=(12, 4))
    plt.imshow(mel_db.squeeze(0).numpy(), aspect='auto', origin='lower', cmap='magma')
    plt.title('Mel-Spectrogram (dB)')
    plt.xlabel('Frames')
    plt.ylabel('Mel bins')
    plt.colorbar()
    plt.tight_layout()
    plt.show()

# 예시(주석 해제)
# plot_wave_and_melspec('tts_ko.wav')



### 5. 마무리 및 과제 제안(업데이트)
- 참조 화자 오디오 1↔2~3개 비교(클로닝 안정성)
- 문장 길이/부호/언어 변화에 따른 억양/자연스러움 비교
- 음성 품질 점검 체크리스트(발음·억양·잡음)
- 확장: 영어 문장에 한국어 억양 유도하기(쉼표/강세 단서 추가)

